In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

## Data Set Information
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

---

## Attribute Information
- **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'C', it indicates a cancellation.
- **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- **Description**: Product (item) name. Nominal.
- **Quantity**: The quantity of each product (item) per transaction. Numeric.
- **InvoiceDate**: Invoice date and time. Numeric, representing the day and time when each transaction was generated.
- **UnitPrice**: Unit price. Numeric, representing the product price per unit in sterling.
- **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- **Country**: Country name. Nominal, representing the name of the country where each customer resides.


In [ ]:
raw_data = pd.read_csv('../data/retail.csv', encoding='ISO-8859-1')
display(raw_data.head(10))

### Transactions

In [ ]:
transactions = raw_data.copy()

transactions = transactions.rename(columns={
    'InvoiceNo': 'invoice_id',
    'StockCode': 'item_id',
    'Description': 'item_description',
    'Quantity': 'quantity_amount',
    'InvoiceDate': 'event_timestamp_invoiced_at',
    'UnitPrice': 'unit_price_eur',
    'CustomerID': 'customer_id',
    'Country': 'country_name'
})

transactions['invoice_id'] = transactions['invoice_id'].astype(str)
transactions['item_id'] = transactions['item_id'].astype(str)
transactions['item_description'] = transactions['item_description'].astype(str)
transactions['quantity_amount'] = transactions['quantity_amount'].astype(int)
transactions['event_timestamp_invoiced_at'] = pd.to_datetime(transactions['event_timestamp_invoiced_at'])
transactions['unit_price_eur'] = transactions['unit_price_eur'].astype(float)
transactions['customer_id'] = transactions['customer_id'].astype('Int64')
transactions['country_name'] = transactions['country_name'].astype(str)
transactions.index.name = 'transaction_id'
transactions.reset_index(inplace=True)
transactions['transaction_id'] = transactions['transaction_id'].astype('Int64')

transactions.head(10)


In [ ]:
transactions['is_return'] = transactions['quantity_amount'] < 0

### Items

In [ ]:
items = transactions.groupby(['item_id', 'item_description']).count()[[]].reset_index()
items.sample(20)

In [ ]:
items['is_operational_item'] = (items['item_id'].str.len() < 5) | (items['item_id'].str.contains('gift', case=False))
items['item_family_id'] = items['item_id'].str.extract(r'^(\d+)', expand=False)
items['item_variant'] = items['item_id'].str.extract(r'(\D+)$', expand=False).fillna('')
items['item_variant'] = np.where(items['item_id'] == items['item_family_id'],  np.nan, items['item_variant'])

items.head()

#### Set items additional info flags

In [ ]:
selected_items = items[items['is_operational_item'] == False]

selected_items['is_all_uppercase'] = selected_items['item_description'].str.isupper()
selected_items['description_len'] = selected_items['item_description'].str.len()
selected_items = selected_items.sort_values(by=['item_id', 'description_len'], ascending=[True, False])
selected_items['description_order'] = selected_items.groupby('item_id').cumcount() + 1

selected_items.head()

In [ ]:
weird_items = selected_items[((selected_items['description_order'] > 1) & (selected_items['description_len'] < 13)) | ~selected_items['is_all_uppercase']]
description_list = weird_items['item_description'].drop_duplicates().tolist()
print(description_list)

is_unknown_item = []
is_error_item = []
is_special_item = []
is_modification_item = []

for description in description_list:
    description_lower = description.lower()
    if any(word in description_lower for word in [
        'check', 'damage', 'wet', 'lost', 'found', 'error', 'wrong', 'faulty', 'mouldy', 'smashed', 'broken', 'crush', 'crack', 'unsaleable', 'missing', 'throw away'
    ]):
        is_error_item.append(description)
    elif any(word in description_lower for word in [
        'dotcom', 'amazon', 'fba', 'ebay', 'john lewis', 'showroom', 'voucher', 'cordial jug', 'website'
    ]):
        is_special_item.append(description)
    elif any(word in description_lower for word in [
        'adjust', 'sample', 'sold as', 'mailout', 'allocate', 'temp', 'credit', 'sale', 're-adjustment', 'label mix up'
    ]):
        is_modification_item.append(description)


is_unknown_item = ['?','??','???','Incorrect stock entry.',"can't find",'nan',None]

is_error_item += [
    'on cargo order', 'test', 'barcode problem', 'mixed up', 'michel oops', 'printing smudges/thrown away', 
    'had been put aside', 'rusty thrown away', 'incorrectly made-thrown away.', 'Breakages', 'counted', 
    'Had been put aside.', 'returned', 'thrown away', 'mystery! Only ever imported 1800', 'Dagamed', 
    'code mix up? 84930', 'Printing smudges/thrown away', 'came coded as 20713', 'incorrect stock entry.',
    "thrown away-can't sell",'Thrown away-rusty','Thrown away.','Given away','historic computer difference?....se',
    'alan hodge cant mamage this section',"thrown away-can't sell.", 'label mix up','sold in set?','mix up with c'
]

is_special_item += [
    'MIA', '?display?', 'Amazon Adjustment', 'Lighthouse Trading zero invc incorr', 
    'Dotcomgiftshop Gift Voucher £100.00', 'sold as set?', 
    'High Resolution Image', 'John Lewis','Bank Charges','Next Day Carriage'
]

is_modification_item += [
    'Adjustment', 'OOPS ! adjustment', 'reverse 21/5/10 adjustment', 'reverse previous adjustment', 
    'marked as 23343', 'incorrectly put back into stock', 'Not rcvd in 10/11/2010 delivery', 'Display', 
    'Had been put aside.',  'sold as set by dotcom', 'add stock to allocate online orders', 
    'allocate stock for dotcom orders ta', 'for online retail orders', 'Marked as 23343'
]

unclassified_items = set(description_list) - set(is_error_item) - set(is_special_item) - set(is_modification_item)

print("is_error_item:", is_error_item)
print("is_special_item:", is_special_item)
print("is_modification_item:", is_modification_item)
print("Unclassified items:", unclassified_items)


In [ ]:
items['is_unknown_item'] = items['item_description'].str.lower().isin(is_unknown_item)
items['is_error_item'] = items['item_description'].str.lower().isin(is_error_item)
items['is_special_item'] = items['item_description'].str.lower().isin(is_special_item)
items['is_modification_item'] = items['item_description'].str.lower().isin(is_modification_item)

items.head(10)

### Set categories with Open AI LLM

#### Set item family with categories

In [ ]:
filtered_items = items[~items['is_operational_item'] & ~items['is_error_item'] & ~items['is_special_item'] & ~items['is_modification_item']& ~items['is_unknown_item']]
grouped = filtered_items.groupby('item_family_id')['item_description'].apply(lambda x: ' '.join(x))

def get_most_common_words(text):
    words = text.split()  
    word_counts = Counter(words)
    max_count = max(word_counts.values(), default=0)
    
    most_common_words = [word for word, count in word_counts.items() if count == max_count]
    return ' '.join(most_common_words) if most_common_words else None

main_product_descriptions = grouped.apply(get_most_common_words).reset_index()
main_product_descriptions.columns = ['item_family', 'item_family_description']


main_product_descriptions.head(10)

In [ ]:
categories = [
    "HOME DECOR",
    "KITCHENWARE",
    "TABLEWARE",
    "GIFT ITEMS",
    "FURNITURE",
    "TOYS",
    "CANDLES",
    "JEWELRY",
    "TEXTILES",
    "GARDENING",
    "BATHROOM",
    "STATIONERY",
    "BAGS",
    "LIGHTING",
    "CRAFTS",
    "HOLIDAY",
    "OUTDOOR",
    "TOOL & DIY",
    "SPORTS",
    "STORAGE",
    "TRAVEL",
    "BEAUTY",
    "BOOKS",
    "GAMES",
    "ORGANIZERS",
    "TABLE LINEN",
    "CHILDREN",
    "CERAMICS",
    "CLOTHING",
    "PLANTS & FLOWERS",
    "OTHERS"
]

#### LLM extraction

In [ ]:
from openai import OpenAI

import openai_setup

# Create GPT Client
def initChatGPTClient():
    organization = openai_setup.conf['organization']
    project = openai_setup.conf['project']
    key = openai_setup.conf['key']

    client = OpenAI(
        api_key=key,
        organization=organization,
        project=project
    )
    return client

def llm_extract_category(client, item):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert classifier system that gives a category to each object from a commerce. Ansewer only with the category selected in uppercase. The possible categories:" + str(categories)},
            {
                "role": "user",
                "content": str(item)
            }
        ]
    )
    answer = completion.choices[0].message.content
    return answer

In [ ]:
client = initChatGPTClient()
main_product_descriptions['category'] = main_product_descriptions['item_family_description'].apply(lambda item: llm_extract_category(client, item))
display(main_product_descriptions)

In [ ]:
display(main_product_descriptions)

In [ ]:
main_product_descriptions.to_csv('../data/main_product_descriptions.csv', index=False)

### Dash ideas

- beneficio mes
- beneficio año
- mom
- yoy

- numero de compras
- numero de clientes
- beneficio total

- por pais

- por categoria de producto

- numero de invoices realizados
- numero de objetos comprados totals
- devoluciones
- flags especiales

- tipos de clientes:
    - gasto
    - numero de invoices realizados
    - numero de objetos comprados totals
    - numero de objetos por compra
    - numero de compras por mes
    - numero de compras por año
    - devoluciones
    - flags especiales

    si lo consigo:
    - categoria que mas compra